# Float BGC Bias Correction, parallel version

Original version: Veronica Tamsitt (USF)

Current authors: Seth Bushinsky, Zachary Nachod (UH Manoa)

Authors: Veronica Tamsitt (USF) et al...

Adapted from MATLAB code written by Seth Bushinsky (UH)

    Download and process GLODAP data
    apply float bias corrections and calculate derivative variables (pH, TALK)
    do float - glodap crossover comparison
    do float - float crossover comparison

Link to MATLAB LIAR/LIPHR code: https://github.com/BRCScienceProducts/LIRs


In [1]:
import xarray as xr
import glob, os
import time
from multiprocessing import Pool
import float_data_processing as fl
import carbon_utils
import numpy as np
import argo_interp_and_crossover as aiac
from gdap_crossover_intermediate_script import process_file


In [15]:
# Set the paths
output_dir = 'output/'
data_dir = 'data/'

#check directories exist
if not os.path.isdir('output'):
    os.mkdir('output')
if not os.path.isdir('data'):
    os.mkdir('data')

# Check for a glodap_offsets_plots directory, create if it does not exist
offset_dir = output_dir + 'glodap_offset_plots/'
if not os.path.isdir(offset_dir):
    os.mkdir(offset_dir)

# read in a user-created text file to point to local directories to avoid having to change this every time 
# we update code
lines=[]
with open('path_file.txt') as f:
    lines = f.readlines()
    
count = 0
for line in lines:
    count += 1
    index = line.find("=")
    #print(f'line {count}: {line}')
    #print(index)
    #print(line[0:index])
    line = line.rstrip()
    if line[0:index].find("argo")>=0:
        argo_path=line[index+1:]
    elif line[0:index].find("liar")>=0:
        liar_dir=line[index+1:]
    elif line[0:index].find("matlab")>=0:
        matlab_dir=line[index+1:]

#add derived float file directory within argo_path
argo_path_derived = argo_path+'../derived_for_crossovers/'
if not os.path.isdir(argo_path_derived):
    os.mkdir(argo_path_derived)



In [18]:
# User definted inputs
#pressure limits for interpolation of 
p_interp_min = 1450 #minimum pressure for float crossover comparison
p_interp_max = 2000 #maximum pressure for float crossover comparison
#pressure levels to interpolate to, every 1db
p_interp = np.arange(p_interp_min,p_interp_max+1)

# select glodap pressure range for comparison
p_compare_min = 1400
p_compare_max = 2100

#max density difference to store crossover
delta_dens = 0.005
# delta_dens = 0.05

#max spice difference to store crossover
delta_spice = 0.005
# delta_spice = 0.05

# max pressure difference to store crossover
# delta_press = 50
delta_press = 100

#crossover distance range
dist = 100

#toggle to plot offsets profile by profile
plot_profile = 0

# when making major changes, list version number here
ver_n = '7' 
# v2 - moving interpolated spice and density calculation to post-PSAL and TEMP interpolation
# v3 - fixed PH_25C calculation for float data, fixed in situ pH comparison (I think)
# v4 - added back in SI and NO3 to DIC calculation - makes a difference apparently (also changes which points have valid data)
# v5 - trying to do near-surface comparisons as well 
# v6 - working on full depth comparison that I will then separate by depth 
# v7 - trying to move code to parallel computing 

glodap_file_offsets_dir = output_dir + 'glodap_file_offsets_' + str(dist) + 'km_' \
    + str(p_compare_min) + '_to_' + str(p_compare_max) + '_' + str(delta_press) + 'm_' + \
    str(delta_dens) + 'dens_' + str(delta_spice) + '_spice' + '_' + ver_n + '/'

if not os.path.isdir(glodap_file_offsets_dir):
    os.mkdir(glodap_file_offsets_dir)

argo_path_interpolated = argo_path+'../interpolated_for_crossovers_' + str(dist) + 'km_' \
    + str(p_compare_min) + '_to_' + str(p_compare_max) + '_' + str(delta_press) + 'm_' + \
    str(delta_dens) + 'dens_' + str(delta_spice) + '_spice' + '_' + ver_n + '/'
if not os.path.isdir(argo_path_interpolated):
    os.mkdir(argo_path_interpolated)


glodap_offsets_filename = 'glodap_offsets_' + str(dist) + 'km_' \
    + str(p_compare_min) + '_to_' + str(p_compare_max) + '_' + str(delta_press) + 'm_' + \
    str(delta_dens) + 'dens_' + str(delta_spice) + '_spice' + '_' + ver_n + '.nc'

In [19]:
# Inputs that usually will not change:

#variables to do crossovers
var_list_plot = ['PRES_ADJUSTED','TEMP_ADJUSTED','PSAL_ADJUSTED','DOXY_ADJUSTED','NITRATE_ADJUSTED',
                 'DIC','pH_25C_TOTAL_ADJUSTED','PH_IN_SITU_TOTAL_ADJUSTED','PDENS']

qc_data_fields = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 
                  'PRES_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED']

bgc_data_fields = ['DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED']

#variables to save to derived file
derived_list = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED',
            'pH_25C_TOTAL_ADJUSTED', 'PDENS', 'spice', 'PRES_ADJUSTED', 'DIC','TALK_LIAR']

# variables to do interpolation on:
interpolation_list = ['TEMP_ADJUSTED', 'PSAL_ADJUSTED', 'DOXY_ADJUSTED', 'NITRATE_ADJUSTED', 'PH_IN_SITU_TOTAL_ADJUSTED',
            'pH_25C_TOTAL_ADJUSTED', 'PRES_ADJUSTED', 'DIC','TALK_LIAR']

## 1. Download and process GLODAP data

In [6]:
gdap = fl.get_glodap(data_dir, year = 2022)
gdap.G2longitude[gdap.G2longitude < 0.] = gdap.G2longitude[gdap.G2longitude < 0.] + 360.
#set flagged data to NaN (is this needed? or masked array better?)
flagvars = ['G2salinity','G2oxygen','G2nitrate','G2tco2','G2talk','G2phts25p0']

for v in flagvars:
    flag = v+'f'
    naninds = gdap[flag]!=2
    gdap[v][naninds] = np.nan

# GLODAP derived variables: density, MLD and pH

#calc potential density
gdap['sigma0_calculated'] = carbon_utils.sigma0(gdap.G2salinity.values,gdap.G2temperature.values,
                                  gdap.G2longitude.values,gdap.G2latitude.values,gdap.G2pressure.values)
#calculate spice
gdap['spice'] = carbon_utils.spiciness0(gdap.G2salinity.values,gdap.G2temperature.values,
                                  gdap.G2longitude.values,gdap.G2latitude.values,gdap.G2pressure.values)

#pH from LIPHR
# calculate LIPHR pH at Glodap points below 1480 m and above 2020m (V: where does the depth restriction come in?)
LIPHR_path = liar_dir
Coordinates = np.stack((gdap.G2longitude.values.flatten(), 
                        gdap.G2latitude.values.flatten(), 
                        gdap.G2pressure.values.flatten()),
                        axis=1)
Measurements = np.stack((gdap.G2salinity.values.flatten(), 
                         gdap.G2temperature.values.flatten(), 
                         gdap.G2oxygen.values.flatten()),
                         axis=1)
MeasIDVec = [1, 7, 6]
                                
results = carbon_utils.LIPHR_matlab(LIPHR_path,
                                    Coordinates.tolist(),
                                    Measurements.tolist(),
                                    MeasIDVec, 
                                    OAAdjustTF = False)                                  

gdap['pH_in_situ_total'] = results
gdap.pH_in_situ_total[np.isnan(gdap.G2phts25p0)] = np.nan
# gdap pH 25C 
gdap['pH_25C_TOTAL_ADJUSTED'] = carbon_utils.co2sys_pH25C(2300.,gdap.pH_in_situ_total,gdap.G2temperature,
                                                         gdap.G2salinity,gdap.G2pressure)
#set pH to nan where there was no original pH data from GLODAP
gdap.pH_25C_TOTAL_ADJUSTED[np.isnan(gdap.G2phts25p0)]=np.nan

#rename GLODAP comparison variables to match argo
gdap = gdap.rename(columns={'G2longitude':'LONGITUDE', 'G2latitude':'LATITUDE', 'G2pressure':'PRES_ADJUSTED',
                            'G2temperature':'TEMP_ADJUSTED','G2salinity':'PSAL_ADJUSTED', 
                            'G2oxygen':'DOXY_ADJUSTED','G2nitrate':'NITRATE_ADJUSTED', 'G2tco2':'DIC', 
                            'G2talk':'TALK_LIAR', 'G2MLD':'MLD','G2o2sat':'o2sat', 'G2PTMP':'PTMP', 
                            'pH_in_situ_total':'PH_IN_SITU_TOTAL_ADJUSTED','sigma0_calculated':'PDENS'})

gdap['obs_index']=gdap.reset_index().index

https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0257247/GLODAPv2.2022_Merged_Master_File.csv


/Users/smb-uh/UHM_Ocean_BGC_Group Dropbox/Seth Bushinsky/Work/Manuscripts/2022_07 Deep O2 Bias/argo_deep_o2_bias/float_data_processing.py:36: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gdap = pd.read_csv(save_dir+'GLODAPv2.'+str(year)+'_Merged_Master_File.csv')
/var/folders/7r/4f_w_nb56llcwh96t_pjq_qh0000gn/T/ipykernel_69135/918574005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdap.G2longitude[gdap.G2longitude < 0.] = gdap.G2longitude[gdap.G2longitude < 0.] + 360.
/var/folders/7r/4f_w_nb56llcwh96t_pjq_qh0000gn/T/ipykernel_69135/918574005.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [61]:
start_time = time.perf_counter()

# 0: overwrites and runs all floats in the argo_path directory 
# 1: reads in and adds to argo_interp_temp.nc rather than overwriting and running all floats
    # need to modify with new parallel version I think 
# 2: runs specific floats listed below
append_data = 0
num_processes = 20  

if 'argo_interp' in locals():
    argo_interp.close()
    
argolist = []

for file in os.listdir(argo_path):
    if file.endswith('Sprof.nc'):
        argolist.append(file)
argolist.sort()

if append_data==1 and os.path.exists(data_dir+'argo_interp_temp.nc'):
    #load previously saved argo_interp
    argo_interp = xr.load_dataset(data_dir+'argo_interp_temp.nc')

    #extract wmo #s as integers from argolist
    s = [int(s) for s in re.findall("[0-9]+", str(argolist))]
    
    #find indices of argolist where argo_interp has no existing matching wmos
    indices = [s.index(i) for i in s if i not in argo_interp.wmo]
    argolist_run = [argolist[i] for i in indices]
elif append_data==0:
    argolist_run=argolist
   
else:
    argolist_run = ['5906547_Sprof.nc',
                        '5906548_Sprof.nc',
                        '5906549_Sprof.nc', 
                        '5906550_Sprof.nc', 
                        '5906551_Sprof.nc', 
                        '5906552_Sprof.nc', 
                        '5906553_Sprof.nc',
                        '5906562_Sprof.nc',
                        '5906554_Sprof.nc',
                        '5906561_Sprof.nc', 
                        '5906556_Sprof.nc',
                        '5906558_Sprof.nc',
                        '5906559_Sprof.nc',
                        '5906557_Sprof.nc']

#restrict glodap data to comparison pressure range
gdap_p = gdap[(gdap.PRES_ADJUSTED.values>p_compare_min) & (gdap.PRES_ADJUSTED.values<p_compare_max)]


if __name__ == "__main__":
    
    # with Pool(processes=num_processes) as pool:
    #     # Create a list of arguments for pool.starmap
    #     argo_args = [(argo_path, liar_dir, argo_path_interpolated, argo_path_derived, file, qc_data_fields, bgc_data_fields, p_interp, derived_list, interpolation_list) for file in argolist_run]
        
    #     # Use pool.starmap with the list of arguments
    #     pool.starmap(aiac.argo_interp_profiles, argo_args)
    
    # interpolated_list = []
    # for file in os.listdir(argo_path_interpolated):
    #     if file.endswith('_interpolated.nc'):
    #         interpolated_list.append(file)
    
    # only run glodap crossovers on floats that have an interpolated file 
    with Pool(processes=num_processes) as pool:
        # Create a list of arguments for pool.starmap
        argo_args = [(argo_path_interpolated, offset_dir, glodap_file_offsets_dir, file, dist, delta_dens, delta_spice, delta_press, \
                        gdap_p, p_interp, plot_profile, var_list_plot, p_compare_min, p_compare_max) for file in argolist_run]
        
        # Use pool.starmap with the list of arguments
        pool.starmap(aiac.glodap_crossover_offsets, argo_args)




# Now load in all individual offset files and concatenate into larger file
crossover_list = []
for file in os.listdir(glodap_file_offsets_dir):
    if file.endswith('_offset.nc'):
        crossover_list.append(file)
# print(len(crossover_list))

if 'glodap_offsets' in locals():
       del glodap_offsets # deletes argo_interp in case this code is being run multiple times. 

for idx, gdap_offset_file in enumerate(crossover_list):
    # print(idx)
    # print(gdap_offset_file)
    gdap_offset_n = xr.open_dataset(glodap_file_offsets_dir + gdap_offset_file)

    if len(gdap_offset_n['N_CROSSOVERS'])>0:
        if 'glodap_offsets' not in locals(): # modified to deal w/ situation where n==0 skipped defining argo_interp
            glodap_offsets = gdap_offset_n
        else:
            glodap_offsets = xr.concat([glodap_offsets,gdap_offset_n],'N_CROSSOVERS')
print(glodap_offsets)

glodap_offsets.to_netcdf(output_dir+glodap_offsets_filename)

print('Total number of glodap crossovers: ' + str(len(glodap_offsets.N_CROSSOVERS)))

finish_time = time.perf_counter()
print("Program finished in {} seconds - using multiprocessing".format(finish_time - start_time))
print("---")

Starting crossover for 1900650
Starting crossover for 1901213
Starting crossover for 1901501
No non-NAN bgc adjusted data for: 1901501
Starting crossover for 1901668
<xarray.Dataset>
Dimensions:                           (N_CROSSOVERS: 2)
Coordinates:
  * N_CROSSOVERS                      (N_CROSSOVERS) int64 0 1
Data variables: (12/44)
    p_compare_min                     int64 1400
    p_compare_max                     int64 2100
    delta_dens                        float64 0.005
    delta_spice                       float64 0.005
    delta_press                       int64 100
    dist                              int64 100
    ...                                ...
    glodap_longitude                  (N_CROSSOVERS) float64 328.3 330.0
    main_float_latitude               (N_CROSSOVERS) float64 53.04 53.33
    glodap_latitude                   (N_CROSSOVERS) float64 53.84 53.42
    glodap_cruise                     (N_CROSSOVERS) int64 46 2104
    glodap_station                

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/smb-uh/UHM_Ocean_BGC_Group Dropbox/Datasets/Data_Products/BGC_ARGO_GLOBAL/2023_06_20/interpolated_for_crossovers_100km_1400_to_2100_100m_0.005dens_0.005_spice_7/1900650_interpolated.nc'

In [45]:
aiac.glodap_crossover_offsets(argo_path_interpolated, offset_dir, glodap_file_offsets_dir, argolist_run[0], dist, delta_dens, delta_spice, delta_press, \
                        gdap_p, p_interp, plot_profile, var_list_plot, p_compare_min, p_compare_max)

Starting crossover for 7901028_glodap_


FileNotFoundError: [Errno 2] No such file or directory: b'/Users/smb-uh/UHM_Ocean_BGC_Group Dropbox/Datasets/Data_Products/BGC_ARGO_GLOBAL/2023_06_20/interpolated_for_crossovers_100km_1400_to_2100_100m_0.005dens_0.005_spice_7/7901028_glodap__interpolated.nc'

In [21]:
# Now load in all individual offset files and concatenate into larger file
interpolated_list = []
for file in os.listdir(argo_path_interpolated):
    if file.endswith('_interpolated.nc'):
        interpolated_list.append(file)
print(len(interpolated_list))

1591


In [22]:
argo_args = [(argo_path_interpolated, offset_dir, glodap_file_offsets_dir, file, dist, delta_dens, delta_spice, delta_press, \
                        gdap_p, p_interp, plot_profile, var_list_plot, p_compare_min, p_compare_max) for file in interpolated_list]
        